In [8]:
import numpy as np
import cv2 as cv


def pre_pro(pic_name, x0, y0):
    # 打开图片
    img = cv.imread(pic_name)
    # 灰度化
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # 二值化
    # 此时的第二个和第三个参数，
    # 即二值化的上下阈值需要根据图片的实际情况个来调整
    ret, binary_img = cv.threshold(gray_img, 180, 255, cv.THRESH_BINARY)
    print("binary_img")
    print(binary_img)
    print(binary_img.shape)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    # 二值图像反相
    con_bi_img = 255 - binary_img
    # print(np.shape(binary_img))
    cv.imshow('con_bi_img', con_bi_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    column_sum_img = np.sum(con_bi_img, axis=0)
    row_sum_img = np.sum(con_bi_img, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print("列：\n")
    print(sort_column_sum[len(sort_column_sum) - 5:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 5:])
    print("行：\n")
    print(sort_row_sum[len(sort_row_sum) - 5:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 5:])

    # 通过每一列和每一行像素和的输出值，判断坐标轴的位置
    # 在坐标轴范围内遍历每一列，将曲线下面的部分设置成黑色
    for i in range(x0, binary_img.shape[1]):
        flag = 0
        for j in range(0, binary_img.shape[0]):
            if binary_img[j][i] == 0:
                flag += j
                break
        for j in range(flag, y0):
            binary_img[j][i] = 0
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()
    return binary_img

In [9]:
import numpy as np
from numpy.core.fromnumeric import shape
import cv2 as cv

def get_points(picture_name, xMax, xMin, yMax, yMin, x0, y0):

    binary_img = pre_pro(picture_name, x0, y0)
    cv.imshow('binary_img', binary_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 边缘提取
    xgrd = cv.Sobel(binary_img, cv.CV_16SC1, 1, 0)
    ygrd = cv.Sobel(binary_img, cv.CV_16SC1, 0, 1)

    egde_output = cv.Canny(xgrd, ygrd, x0, y0)
    cv.imshow('canny_edge', egde_output)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 图像像素按行和列求和
    column_sum_img = np.sum(egde_output, axis=0)
    row_sum_img = np.sum(egde_output, axis=1)
    # 排序
    sort_column_sum = np.sort(column_sum_img)
    sort_column_sum_indices = np.argsort(column_sum_img)
    sort_row_sum = np.sort(row_sum_img)
    sort_row_sum_indices = np.argsort(row_sum_img)
    print(sort_column_sum[len(sort_column_sum) - 10:])
    print(sort_column_sum_indices[len(sort_column_sum_indices) - 10:])
    print(sort_row_sum[len(sort_row_sum) - 10:])
    print(sort_row_sum_indices[len(sort_row_sum_indices) - 10:])

    fc = egde_output[0:binary_img.shape[0], 0:binary_img.shape[1]]
    cv.imshow('function_curve', fc)
    cv.waitKey(0)
    cv.destroyAllWindows()

    # 提取图像数据
    rows = (fc.shape)[0]
    cols = (fc.shape)[1]

    min_x = xMin
    max_x = xMax
    min_y = yMin
    max_y = yMax

    x_axis = np.empty([rows, cols])
    y_axis = np.empty([cols, rows])

    # x_interval和y_interval用于调整数据长度，在报错的时候可以通过他们来调整
    x_interval = (max_x - min_x) / (cols + 1)
    x_value = np.arange(min_x + x_interval, max_x, x_interval)
    # 自动调整长度
    if(cols != (x_value.shape)[0]):
        x_interval = (max_x - min_x) / (cols)
        x_value = np.arange(min_x + x_interval, max_x, x_interval)
    y_interval = (max_y - min_y) / (rows + 1)
    y_value = np.arange(max_y - y_interval, min_y, -y_interval)
    if(rows != (y_value.shape)[0]):
        y_interval = (max_y - min_y) / (rows)
        y_value = np.arange(max_y - y_interval, min_y, -y_interval)

    x_axis[:, ] = x_value
    y_axis[:, ] = y_value
    y_axis = y_axis.T

    x_fc = x_axis.T[fc.T == 255]
    y_fc = y_axis.T[fc.T == 255]

    return (x_fc, y_fc)

In [10]:
# generate excel file
import xlsxwriter
import numpy as np 


#函数模板
# def get_points(picture_name, xMax, xMin, yMax, yMin, x0, y0):
# pic_name = "demo03(2).png"
# pic_name_original = 'demo03.png'
# res = get_points(pic_name, 2100, 2015, 160, -20, 15, 233)

pic_name = "15_Extreme weather events - ps-x0-82-y0-42(2).png"
pic_name_original = '15_Extreme weather events - ps-x0-82-y0-42.png'
res = get_points(pic_name, 1880, 2020, 60, 150, 0, 0)

book = xlsxwriter.Workbook(f"{pic_name_original.split('.')[0]}.xlsx")
sheet = book.add_worksheet(name='data');
sheet.write(0, 0, "x_axis");
sheet.write(0, 1, "y_axis");

xValue = -999999;
row = 1;
points = np.array(res);
[rows, cols] = points.shape
for j in range(cols):
  if np.floor(points[0][j]) > xValue:
    sheet.write(row, 0, np.floor(points[0][j]));
    sheet.write(row, 1, points[1][j]);
    xValue = np.floor(points[0][j]);
    row += 1;

sheet = book.add_worksheet(name = 'A1');
sheet.insert_image(pic_name);
sheet = book.add_worksheet(name = 'A1_original');
sheet.insert_image(pic_name_original);
book.close();

binary_img
[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
(369, 593)
列：

[19125 19125 21930 24480 24735]
[ 23 535  22 537 536]
行：

[42840 42840 42840 83895 84405]
[206 207 258 293 336]
[ 7905  9690  9945 10200 10710 10965 11220 11475 11475 15810]
[ 24  21 535  22 534 539 533 538 537 536]
[ 42075  42075  44880  45135  97410  97665 145095 145095 145605 145605]
[259 204 293 336 250 252 292 294 335 337]
